# GNNS JSON-file Decoder

In [1]:

import numpy as np
from io import BytesIO
from scipy.interpolate import CubicSpline
import json
from datetime import datetime
from matplotlib import pyplot as plt




In [52]:
import base64
import io
from PIL import Image


with open("recordings/tst.json", 'r') as file:
    data = json.load(file)


base64_string = data[0]['detectedImage']

decoded_data = base64.b64decode(base64_string)
# Convert raw pixel data to numpy array
img_array = np.frombuffer(decoded_data, dtype=np.uint8)
# Reshape the array into an image
image = Image.fromarray(img_array)
# Show the image
image.show()

print(base64_string)

'''
print(base64_string)
str = "data:image/png;base64,"+base64_string
decoded_data = base64.b64decode(str)
byte_image = Image.open(io.BytesIO(decoded_data))
byte_image.show()

'''


...


'\nprint(base64_string)\nstr = "data:image/png;base64,"+base64_string\ndecoded_data = base64.b64decode(str)\nbyte_image = Image.open(io.BytesIO(decoded_data))\nbyte_image.show()\n\n'

In [ ]:

'''
# Assuming the dimensions of your image, replace these values with actual dimensions
# For example, if your image is 100x100 pixels, replace (100, 100) with (100, 100, 3)
image_shape = (720,1208, 3)

# Reshape the array to match the image dimensions

image_array = image_array.reshape(image_shape)


# Plot the image
plt.imshow(image_array)
plt.axis('off')  # Optional: turn off axis
plt.show()
'''


In [3]:
def read_json(filename: str):
    '''Reads coordinates from a json file and returns them in there original structure'''
    with open(filename, 'r') as file:
        data = json.load(file)
    return data['startTime'], data['stopTime'], data['coordinates']

def transform_from_nmea_coordinates_to_degrees(latitude_ddmm, longitude_ddmm) -> tuple[float, float]: 
    '''Converting the coordinates from nmea format (ddmm.mm) to degreees format (D.d)'''
    # Latitude conversion
    lat_degrees = int(latitude_ddmm // 100)  # Extracting degrees
    lat_minutes = latitude_ddmm % 100  # Extracting minutes
    latitude_decimal = lat_degrees + lat_minutes / 60  # Convert to decimal degrees

    # Longitude conversion
    lon_degrees = int(longitude_ddmm // 100)  # Extracting degrees
    lon_minutes = longitude_ddmm % 100  # Extracting minutes
    longitude_decimal = lon_degrees + lon_minutes / 60  # Convert to decimal degrees
    return latitude_decimal, longitude_decimal

def seperate_latitude_longitude_timestamps(coordinates: list):
    latitudes = []
    longitudes = []
    timestamps = []
    for c in coordinates:
        latitudes.append(c['lat'])
        longitudes.append(c['long'])
        timestamps.append(c['timeStamp'])
    return latitudes, longitudes, timestamps


def convert_timestamp_to_seconds(timestamp: str, starttime: str, utcDifference = 1):
    # Convert 'hhmmss.mm' format to seconds
    start_date = datetime.strptime(starttime, "%Y-%m-%dT%H:%M:%S.%f")

    hours = int(timestamp[:2]) - start_date.hour + utcDifference
    minutes = int(timestamp[2:4]) - start_date.minute
    seconds = int(timestamp[4:6]) - start_date.second
    centiseconds = int(timestamp[7:]) - round(start_date.microsecond/10000)
    total_seconds = hours * 3600 + minutes * 60 + seconds + centiseconds / 100
    return total_seconds


def interpolate_coordinates_linear(latitudes, longitudes, timestamps, starttime):
    # Convert strings to floats
    latitudes = [float(lat) for lat in latitudes]
    longitudes = [float(lon) for lon in longitudes]
    timestamps = [convert_timestamp_to_seconds(timestamp, starttime) for timestamp in timestamps]

    # Convert timestamps to milliseconds
    timestamps_ms = [timestamp * 1000 for timestamp in timestamps]

    # Create a linearly spaced array for each millisecond
    interpolated_timestamps = np.arange(timestamps_ms[0], timestamps_ms[-1] + 1, 1)

    # Interpolate latitude and longitude
    interpolated_latitudes = np.interp(interpolated_timestamps, timestamps_ms, latitudes)
    interpolated_longitudes = np.interp(interpolated_timestamps, timestamps_ms, longitudes)

    # Convert interpolated timestamps back to seconds
    interpolated_timestamps /= 1000

    return interpolated_latitudes, interpolated_longitudes, interpolated_timestamps


def interpolate_coordinates_smooth(latitudes, longitudes, timestamps, starttime):
    # Convert strings to floats
    latitudes = np.array([float(lat) for lat in latitudes])
    longitudes = np.array([float(lon) for lon in longitudes])
    timestamps = np.array([convert_timestamp_to_seconds(timestamp, starttime) for timestamp in timestamps])

    # Convert timestamps to milliseconds
    timestamps_ms = timestamps * 1000

    # Create a cubic spline interpolation for latitude and longitude
    spline_lat = CubicSpline(timestamps_ms, latitudes)
    spline_lon = CubicSpline(timestamps_ms, longitudes)

    # Interpolate at a higher resolution
    interpolated_timestamps_ms = np.arange(timestamps_ms[0], timestamps_ms[-1] + 1, 1)
    interpolated_latitudes = spline_lat(interpolated_timestamps_ms)
    interpolated_longitudes = spline_lon(interpolated_timestamps_ms)

    # Convert interpolated timestamps back to seconds
    interpolated_timestamps = interpolated_timestamps_ms / 1000

    return interpolated_latitudes, interpolated_longitudes, interpolated_timestamps


def get_position_at_time(latitudes:list, longitudes:list, timestamps:list, target_time:str, starttime: str, utcDifference=0):
    # Interpolate coordinates
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_linear(
        latitudes, longitudes, timestamps, starttime) 
    # Find the index of the target time in the interpolated timestamps
    target_time_seconds = convert_timestamp_to_seconds(target_time, starttime, utcDifference)
    index = np.searchsorted(interpolated_timestamps, target_time_seconds)
    # Return the corresponding latitude and longitude
    return interpolated_latitudes[index], interpolated_longitudes[index]

def get_position_at_seconds(latitudes:list, longitudes:list, timestamps:list, target_seconds:float, starttime: str):
    # Interpolate coordinates
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_linear(
        latitudes, longitudes, timestamps, starttime)
    # Find the index of the target time in the interpolated timestamps
    index = np.searchsorted(interpolated_timestamps, target_seconds)
    # Return the corresponding latitude and longitude
    return interpolated_latitudes[index], interpolated_longitudes[index]

def write_smooth_geojson_file(input_filename: str, output_filename: str):
    starttime, stoptime, coordinates = read_json(input_filename)
    latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_smooth(latitudes, longitudes, timestamps)
    interpolated_coordinates = []
    for i in range(len(interpolated_timestamps)):
        i_lat = interpolated_latitudes[i]
        i_long = interpolated_longitudes[i]
        i_lat_dec, i_long_dec = transform_from_nmea_coordinates_to_degrees(i_lat, i_long)
        cord = [i_long_dec, i_lat_dec]
        interpolated_coordinates.append(cord)
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "LineString",
            "coordinates": interpolated_coordinates
        }
    }
    with open(output_filename, 'w') as file:
        json.dump(feature, file)

def write_linear_geojson_file(input_filename: str, output_filename: str):
    starttime, stoptime, coordinates = read_json(input_filename)
    latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_linear(latitudes, longitudes, timestamps, starttime)
    interpolated_coordinates = []
    for i in range(len(interpolated_timestamps)):
        i_lat = interpolated_latitudes[i]
        i_long = interpolated_longitudes[i]
        i_lat_dec, i_long_dec = transform_from_nmea_coordinates_to_degrees(i_lat, i_long)
        cord = [i_long_dec, i_lat_dec]
        interpolated_coordinates.append(cord)
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "LineString",
            "coordinates": interpolated_coordinates
        }
    }
    with open(output_filename, 'w') as file:
        json.dump(feature, file)


## Examples of usage

In [4]:
starttime, stoptime, coordinates = read_json("recordings/buss.json") # recording from 105033.00 to 105859.75
latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)

### Get coordinate from target time

In [5]:
target_time = "105044.66"

lat_at_time, long_at_time = get_position_at_time(latitudes, longitudes, timestamps, target_time, starttime, utcDifference=1)
print(f"NMEA format: lat - {lat_at_time}, long - {long_at_time}")

lat_at_time_in_deg, long_at_time_in_deg = transform_from_nmea_coordinates_to_degrees(lat_at_time, long_at_time)
print(f"Degrees format: lat - {lat_at_time_in_deg}, long - {long_at_time_in_deg}")

NMEA format: lat - 6325.62092506, long - 1023.585220712
Degrees format: lat - 63.427015417666674, long - 10.393087011866667


### Get coordinate form stoptime

In [12]:
stoptime_date = datetime.strptime(stoptime, "%Y-%m-%dT%H:%M:%S.%f")
formatted_time = stoptime_date.strftime('%H%M%S.%f')[:-4]  #


print(formatted_time)

lat_at_time, long_at_time = get_position_at_time(latitudes, longitudes, timestamps, formatted_time, starttime, utcDifference=-1)
print(f"NMEA format: lat - {lat_at_time}, long - {long_at_time}")


lat_at_time_in_deg, long_at_time_in_deg = transform_from_nmea_coordinates_to_degrees(lat_at_time, long_at_time)
print(f"Degrees format: lat - {lat_at_time_in_deg}, long - {long_at_time_in_deg}")

115900.68


IndexError: index 506751 is out of bounds for axis 0 with size 506751

### Get coordinate from target seconds

In [ ]:
target_seconds = 188.49
lat_at_sec, long_at_sec = get_position_at_seconds(latitudes, longitudes, timestamps, target_seconds, starttime)
print(f"NMEA format: lat - {lat_at_sec}, long - {long_at_sec}")


lat_at_sec_in_deg, long_at_sec_in_deg = transform_from_nmea_coordinates_to_degrees(lat_at_time, long_at_time)
print(f"Degrees format: lat - {lat_at_sec_in_deg}, long - {long_at_sec_in_deg}")

NMEA format: lat - 6325.303782948, long - 1024.035676484
Degrees format: lat - 63.42818294666666, long - 10.392780253333333


### Write all interpolated coordinates to a GeoJSON file

In [ ]:
write_linear_geojson_file("recordings/buss.json", "recordings/buss_o.json")
